In [1]:
import  torch, os
import  numpy as np
from    subgraph_data_processing import Subgraphs
import  scipy.stats
from    torch.utils.data import DataLoader
from    torch.optim import lr_scheduler
import  random, sys, pickle
import  argparse

import networkx as nx
import numpy as np
from scipy.special import comb
from itertools import combinations 
import networkx.algorithms.isomorphism as iso
from tqdm import tqdm
import dgl

from meta import Meta
import time
import copy
import psutil
from memory_profiler import memory_usage

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
def collate(samples):
        graphs_spt, labels_spt, graph_qry, labels_qry, center_spt, center_qry, nodeidx_spt, nodeidx_qry, support_graph_idx, query_graph_idx = map(list, zip(*samples))

        return graphs_spt, labels_spt, graph_qry, labels_qry, center_spt, center_qry, nodeidx_spt, nodeidx_qry, support_graph_idx, query_graph_idx


In [3]:


argparser = argparse.ArgumentParser()
argparser.add_argument('--epoch', type=int, help='epoch number', default=10)
argparser.add_argument('--n_way', type=int, help='n way', default=3)
argparser.add_argument('--k_spt', type=int, help='k shot for support set', default=3)
argparser.add_argument('--k_qry', type=int, help='k shot for query set', default=24)
argparser.add_argument('--task_num', type=int, help='meta batch size, namely task num', default=32)
argparser.add_argument('--meta_lr', type=float, help='meta-level outer learning rate', default=1e-3)
argparser.add_argument('--update_lr', type=float, help='task-level inner update learning rate', default=1e-2)
argparser.add_argument('--update_step', type=int, help='task-level inner update steps', default=10)
argparser.add_argument('--update_step_test', type=int, help='update steps for finetunning', default=20)
argparser.add_argument('--input_dim', type=int, help='input feature dim', default=1)
argparser.add_argument('--hidden_dim', type=int, help='hidden dim', default=256)
argparser.add_argument('--attention_size', type=int, help='dim of attention_size', default=32)
argparser.add_argument("--data_dir", default="D:\pythonProject\python_file\Graph_DD\G-Meta-master\arxiv", type=str, required=False, help="The input data dir.")
argparser.add_argument("--no_finetune", default=True, type=str, required=False, help="no finetune mode.")
argparser.add_argument("--task_setup", default='Disjoint', type=str, required=False, help="Select from Disjoint or Shared Setup. For Disjoint-Label, single/multiple graphs are both considered.")
argparser.add_argument("--method", default='G-Meta', type=str, required=False, help="Use G-Meta")
argparser.add_argument('--task_n', type=int, help='task number', default=1)
argparser.add_argument("--task_mode", default='False', type=str, required=False, help="For Evaluating on Tasks")
argparser.add_argument("--val_result_report_steps", default=100, type=int, required=False, help="validation report")
argparser.add_argument("--train_result_report_steps", default=200, type=int, required=False, help="training report")
argparser.add_argument("--num_workers", default=0, type=int, required=False, help="num of workers")
argparser.add_argument("--batchsz", default=10000, type=int, required=False, help="batch size")
argparser.add_argument("--link_pred_mode", default='False', type=str, required=False, help="For Link Prediction")
argparser.add_argument("--h", default=2, type=int, required=False, help="neighborhood size")
argparser.add_argument('--sample_nodes', type=int, help='sample nodes if above this number of nodes', default=1000)

args = argparser.parse_args([])



In [4]:
# 访问解析后的参数
print(args.epoch)  # 示例：打印 epoch 参数的值

10


In [5]:
mem_usage = memory_usage(-1, interval=.5, timeout=1)
# 这行代码用于获取当前Python程序的内存使用情况。

# memory_usage() 是一个函数，接受三个参数：

# 第一个参数 -1 表示获取当前程序的内存使用情况。
# 第二个参数 interval=.5 表示每隔0.5秒测量一次内存使用情况。
# 第三个参数 timeout=1 表示测量内存使用情况的总时长为1秒。
# mem_usage 是一个变量，用于存储测量到的内存使用情况。

# 通过这行代码，你可以获取当前Python程序在一段时间内的内存使用情况，以便进一步分析和优化程序的内存消耗
torch.manual_seed(222)
torch.cuda.manual_seed_all(222)
np.random.seed(222)

print(args)

Namespace(epoch=10, n_way=3, k_spt=3, k_qry=24, task_num=32, meta_lr=0.001, update_lr=0.01, update_step=10, update_step_test=20, input_dim=1, hidden_dim=256, attention_size=32, data_dir='D:\\pythonProject\\python_file\\Graph_DD\\G-Meta-master\x07rxiv', no_finetune=True, task_setup='Disjoint', method='G-Meta', task_n=1, task_mode='False', val_result_report_steps=100, train_result_report_steps=200, num_workers=0, batchsz=10000, link_pred_mode='False', h=2, sample_nodes=1000)


In [6]:
root = "D:/pythonProject/python_file/Graph_DD/G-Meta-master/DATA/arxiv/"

In [7]:
feat = np.load( 'D:/pythonProject/python_file/Graph_DD/G-Meta-master/DATA/arxiv/features.npy', allow_pickle = True)

In [8]:
with open('D:/pythonProject/python_file/Graph_DD/G-Meta-master/DATA/arxiv/graph_dgl.pkl', 'rb') as f:
    dgl_graph = pickle.load(f)


In [9]:
with open('D:/pythonProject/python_file/Graph_DD/G-Meta-master/DATA/arxiv/label.pkl', 'rb') as f:
    info = pickle.load(f)

In [10]:
total_class = len(np.unique(np.array(list(info.values()))))
print('There are {} classes '.format(total_class))

There are 40 classes 


In [11]:
if args.task_setup == 'Disjoint':
    labels_num = args.n_way

In [12]:
if len(feat.shape) == 2:
    # single graph, to make it compatible to multiple graph retrieval.
    feat = [feat]  

In [13]:
config = [('GraphConv', [feat[0].shape[1], args.hidden_dim])]

if args.h > 1:
    config = config + [('GraphConv', [args.hidden_dim, args.hidden_dim])] * (args.h - 1)

config = config + [('Linear', [args.hidden_dim, labels_num])]

if args.link_pred_mode == 'True':
    config.append(('LinkPred', [True]))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

maml = Meta(args, config).to(device)

tmp = filter(lambda x: x.requires_grad, maml.parameters())
num = sum(map(lambda x: np.prod(x.shape), tmp))
print(maml)
print('Total trainable tensors:', num)

Meta(
  (net): Classifier(
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 128x256 (GPU 0)]
        (1): Parameter containing: [torch.float32 of size 256 (GPU 0)]
        (2): Parameter containing: [torch.float32 of size 256x256 (GPU 0)]
        (3): Parameter containing: [torch.float32 of size 256 (GPU 0)]
        (4): Parameter containing: [torch.float32 of size 3x256 (GPU 0)]
        (5): Parameter containing: [torch.float32 of size 3 (GPU 0)]
    )
  )
)
Total trainable tensors: 99587


In [14]:
dgl_graph = [dgl_graph]

In [15]:
dgl_graph

[Graph(num_nodes=169343, num_edges=1166243,
       ndata_schemes={}
       edata_schemes={})]

In [16]:

max_acc = 0
model_max = copy.deepcopy(maml)

db_train = Subgraphs(root, 'train', info, n_way=args.n_way, k_shot=args.k_spt, k_query=args.k_qry, batchsz=args.batchsz, args = args, adjs = dgl_graph, h = args.h)
db_val = Subgraphs(root, 'val', info, n_way=args.n_way, k_shot=args.k_spt,k_query=args.k_qry, batchsz=100, args = args, adjs = dgl_graph, h = args.h)
db_test = Subgraphs(root, 'test', info, n_way=args.n_way, k_shot=args.k_spt,k_query=args.k_qry, batchsz=100, args = args, adjs = dgl_graph, h = args.h)
print('------ Start Training ------')

shuffle DB :train, b:10000, 3-way, 3-shot, 24-query, 2-hops
[['0_0', '0_7', '0_30', '0_53', '0_85', '0_112', '0_125', '0_134', '0_162', '0_215', '0_284', '0_292', '0_294', '0_319', '0_327', '0_343', '0_395', '0_396', '0_411', '0_413', '0_414', '0_428', '0_480', '0_483', '0_494', '0_503', '0_507', '0_511', '0_513', '0_536', '0_607', '0_632', '0_653', '0_657', '0_670', '0_674', '0_734', '0_744', '0_758', '0_784', '0_846', '0_896', '0_982', '0_1000', '0_1020', '0_1028', '0_1076', '0_1105', '0_1144', '0_1162', '0_1168', '0_1196', '0_1277', '0_1287', '0_1290', '0_1306', '0_1377', '0_1394', '0_1428', '0_1490', '0_1510', '0_1521', '0_1543', '0_1544', '0_1546', '0_1552', '0_1574', '0_1602', '0_1625', '0_1652', '0_1677', '0_1748', '0_1758', '0_1772', '0_1809', '0_1833', '0_1892', '0_1918', '0_1953', '0_1972', '0_1980', '0_2016', '0_2069', '0_2082', '0_2101', '0_2107', '0_2237', '0_2254', '0_2303', '0_2332', '0_2359', '0_2419', '0_2422', '0_2438', '0_2459', '0_2470', '0_2511', '0_2518', '0_2563'

[['0_6', '0_100', '0_219', '0_310', '0_448', '0_454', '0_741', '0_897', '0_988', '0_996', '0_1167', '0_1185', '0_1197', '0_1489', '0_1658', '0_1660', '0_1673', '0_1926', '0_2390', '0_2428', '0_2602', '0_2834', '0_2982', '0_3050', '0_3086', '0_3199', '0_3327', '0_3532', '0_3607', '0_3657', '0_3678', '0_3683', '0_3692', '0_3723', '0_3842', '0_3843', '0_3900', '0_3918', '0_4309', '0_4332', '0_4356', '0_4363', '0_4436', '0_4567', '0_4591', '0_4593', '0_4853', '0_4968', '0_4985', '0_5034', '0_5176', '0_5429', '0_5523', '0_5548', '0_5592', '0_5604', '0_5625', '0_5861', '0_5877', '0_5882', '0_6148', '0_6305', '0_6362', '0_6449', '0_6518', '0_6698', '0_6920', '0_7031', '0_7113', '0_7118', '0_7152', '0_7355', '0_7486', '0_7643', '0_7809', '0_7839', '0_7849', '0_7940', '0_8094', '0_8095', '0_8103', '0_8126', '0_8286', '0_8308', '0_8454', '0_8888', '0_9032', '0_9084', '0_9091', '0_9109', '0_9341', '0_9349', '0_9449', '0_9569', '0_9579', '0_9761', '0_9768', '0_10109', '0_10122', '0_10344', '0_1035

[['0_5', '0_16', '0_26', '0_67', '0_88', '0_92', '0_106', '0_126', '0_167', '0_177', '0_204', '0_218', '0_249', '0_308', '0_315', '0_318', '0_320', '0_356', '0_378', '0_427', '0_455', '0_464', '0_538', '0_550', '0_659', '0_717', '0_773', '0_783', '0_808', '0_809', '0_852', '0_858', '0_869', '0_871', '0_893', '0_899', '0_912', '0_920', '0_925', '0_954', '0_959', '0_991', '0_1003', '0_1010', '0_1014', '0_1027', '0_1031', '0_1032', '0_1043', '0_1063', '0_1083', '0_1104', '0_1123', '0_1183', '0_1184', '0_1198', '0_1200', '0_1228', '0_1245', '0_1279', '0_1315', '0_1317', '0_1321', '0_1373', '0_1388', '0_1398', '0_1411', '0_1418', '0_1468', '0_1526', '0_1553', '0_1570', '0_1638', '0_1656', '0_1679', '0_1689', '0_1692', '0_1698', '0_1703', '0_1704', '0_1706', '0_1713', '0_1723', '0_1729', '0_1765', '0_1780', '0_1787', '0_1788', '0_1829', '0_1831', '0_1862', '0_1869', '0_1874', '0_1884', '0_1948', '0_1951', '0_1973', '0_1977', '0_2017', '0_2067', '0_2120', '0_2130', '0_2136', '0_2167', '0_2173

In [17]:
s_start = time.time()
max_memory = 0

In [18]:
for epoch in range(args.epoch):
    db = DataLoader(db_train, args.task_num, shuffle=True, num_workers=args.num_workers, pin_memory=True, collate_fn = collate)
    s_f = time.time()
    for step, (x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry) in enumerate(db):
        nodes_len = 0
        if step >= 1:
            data_loading_time = time.time() - s_r
        else:
            data_loading_time = time.time() - s_f
        s = time.time()
        # x_spt: a list of #task_num tasks, where each task is a mini-batch of k-shot * n_way subgraphs
        # y_spt: a list of #task_num lists of labels. Each list is of length k-shot * n_way int.                
        nodes_len += sum([sum([len(j) for j in i]) for i in n_spt])
        accs = maml(x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry, feat)
        max_memory = max(max_memory, float(psutil.virtual_memory().used/(1024**3)))
        if step % args.train_result_report_steps == 0:
            print('Epoch:', epoch + 1, ' Step:', step, ' training acc:', str(accs[-1])[:5], ' time elapsed:', str(time.time() - s)[:5], ' data loading takes:', str(data_loading_time)[:5], ' Memory usage:', str(float(psutil.virtual_memory().used/(1024**3)))[:5])
        s_r = time.time()

    # validation per epoch
    db_v = DataLoader(db_val, 1, shuffle=True, num_workers=args.num_workers, pin_memory=True, collate_fn = collate)
    accs_all_test = []

    for x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry in db_v:

        accs = maml.finetunning(x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry, feat)
        accs_all_test.append(accs)

    accs = np.array(accs_all_test).mean(axis=0).astype(np.float16)
    print('Epoch:', epoch + 1, ' Val acc:', str(accs[-1])[:5])
    if accs[-1] > max_acc:
        max_acc = accs[-1]
        model_max = copy.deepcopy(maml)

D:\miniconda3\lib\site-packages\dgl\backend\pytorch\tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


Epoch: 1  Step: 0  training acc: Param  time elapsed: 7.603  data loading takes: 45.35  Memory usage: 11.54


KeyboardInterrupt: 